I used [this resource](https://nextjournal.com/schmudde/ml4a-mnist) to learn Convolutional Neural Network on the MNIST challenge 

Try google [Colab](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true)

In [2]:
#This is the Kaggle challenge
#check current directory
%cd
#change working directory where the file is
%cd ~\Desktop\Book_ML_deepL

C:\Users\EAMC
C:\Users\EAMC\Desktop\Book_ML_deepL


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import csv as csv

In [3]:
##manual download, then use pd
test_mnist= pd.read_csv('test_MNIST.csv', sep=',')
train_mnist = pd.read_csv('train_MNIST.csv', sep=',')

In [14]:
#from keras.datasets import mnist
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 13s 1us/step


In [30]:
# split train - test and convert to numpy arrays with .values
X_train, y_train= train_mnist.iloc[:, 1:].values, train_mnist.iloc[:, 0].values

In [31]:
print(X_train.shape)
print(y_train.shape)

(42000, 784)
(42000,)


In [35]:
n_train = 42000

In [39]:
from keras.utils.np_utils import to_categorical
# we have to preprocess the data into the right shape
X_train = X_train.reshape(n_train, 28, 28, 1).astype('float32')
# normalize from [0, 255] to [0, 1]
X_train /= 255
# numbers 0-9, so ten classes
n_classes = 10
# convert integer labels into one-hot vectors
y_train = to_categorical(y_train, n_classes)

In [42]:
from keras.models import Sequential
model = Sequential()

In [43]:
# number of convolutional filters
n_filters = 32
# convolution filter size
# i.e. we will use a n_conv x n_conv filter
n_conv = 3
# pooling window size
# i.e. we will use a n_pool x n_pool pooling window
n_pool = 2

In [45]:
from keras.layers import Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D

model.add(Convolution2D(n_filters, kernel_size=(n_conv, n_conv),
        # we have a 28x28 single channel (grayscale) image so the input shape should be (28, 28, 1)
        input_shape=(28, 28, 1)))

model.add(Activation('relu'))

model.add(Convolution2D(n_filters, kernel_size=(n_conv, n_conv)))
model.add(Activation('relu'))

# then we apply pooling to summarize the features
# extracted thus far
model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

In [46]:
from keras.layers import Dropout, Flatten, Dense

model.add(Dropout(0.25))

# flatten the data for the 1D layers
model.add(Flatten())

# Dense(n_outputs)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# the softmax output layer gives us a probablity for each class
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [47]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [49]:
# how many examples to look at during each update step
batch_size = 128

# how many times to run through the full set of examples
n_epochs = 5

# the training may be slow depending on your computer
model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs)

see [here](https://www.kaggle.com/snbhanja/mnist-keras)
and [here](https://hackernoon.com/keras-with-gpu-on-amazon-ec2-a-step-by-step-instruction-4f90364e49ac)

and [here](http://liufuyang.github.io/2017/04/02/just-another-tensorflow-beginner-guide-4.html)

# Submit

In [51]:
X_test = test_mnist.values.astype('float32')

In [ ]:
# prediction
yPred = model.predict_classes(X_test,verbose=1)
#convert to integer
yPred = yPred.astype(int)
#create a series with the predictions
results = pd.Series(yPred,name="Label")
# then a second series with the index from 1 to x
index= pd.Series(range(1,28001), name = "ImageId")
#join the two series into a single dataframe
submission = pd.concat([index, results], axis = 1)
#save results to csv
submission.to_csv("CNN_mnist.csv",index=False)